In [2]:
import pandas as pd 
cleaned_customers = pd.read_csv(r"C:\Users\Dhivakar\pipelines\venv\CSV\cleaned_customer_data2.csv")
cleaned_customers = cleaned_customers.head(10)
cleaned_customers

,customer_id,name,email,phone,address,registration_date,loyalty_status,country_code
0,1,Michelle Kidd,vayala@example.net,+971 6197234258,"USNS Santiago, FPO AE 80872",25-01-2025,Gold,AE
1,2,Brad Newton,taylorcatherine@example.net,+1 5376741158,"38783 Oliver Street, West Kristenborough, MT 9...",13-07-2023,Silver,MT
2,3,Larry Torres,dsanchez@example.net,+1 8102564505,"6845 Steele Turnpike, West Erikabury, UT 37487",18-08-2023,Bronze,UT
3,4,Kimberly Price,jessicaknight@example.com,+1 4232229779,"1631 Alexis Meadows, Lake Amanda, CA 75179",08-12-2024,Gold,CA
4,5,Matthew Phillips,qwilliams@example.com,+1 2207633522,"2274 Williams Heights Suite 895, Andersonhaven...",03-02-2024,Gold,OR
5,6,Darrell Gallagher Dvm,larryharris@example.org,+1 3799503695,"19887 Brandon Bridge, New Rebecca, MA 94947",03-03-2025,Bronze,MA
6,7,Louis Miller,patriciaelliott@example.org,+1 1988627105,"02590 Marshall Well, Sheppardland, CT 88067",30-09-2024,Gold,CT
7,8,Melissa Galloway Md,watsonbenjamin@example.net,+1 5485499586,"209 Steven Oval Apt. 022, Mollyhaven, NC 10211",06-10-2023,Bronze,NC
8,9,Matthew Harvey,bgaines@example.net,+1 5811447338,"60123 Davis Ford, Mistychester, WA 69400",31-10-2024,Silver,WA
9,10,Rachel White,whitemichael@example.org,+1 4107299740,"6182 Brown Mountain, South Gary, IL 29016",25-02-2024,Bronze,IL


In [3]:
#connecting with database
import pyodbc

conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=DESKTOP-29912U2\MSSQLSERVER1;'
    'DATABASE=Scdpipelines;'
    'UID=sa;'
    'PWD=Dhiva@2003;'
    'TrustServerCertificate=yes;'
)

cursor = conn.cursor()
 
print("Database Connected successfully!")

Database Connected successfully!


In [ ]:
#Loading Data into Database.
from sqlalchemy.engine import URL
from sqlalchemy import create_engine
import pandas as pd
 

connection_url = URL.create(
    "mssql+pyodbc",
    username="sa",
    password="Dhiva@2003",
    host="DESKTOP-29912U2\MSSQLSERVER1",
    database="Scdpipelines",
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "TrustServerCertificate": "yes"
    }
)
 
engine = create_engine(connection_url)

cleaned_customers.to_sql("customers1", con=engine, if_exists="replace", index=False)

 
print("Data uploaded successfully to SQL Server!")
 


In [14]:
#Fetched first 10 rows from the cleaned customer table and start to perform SCD.
#SCD1 

cursor.execute("""
 Update customers1
 Set email = 'bradnewton@example.com'
 where customer_id = 2
""")

conn.commit()

In [15]:
#SCD2 

cursor.execute ("""
Alter table customers1
add end_registration_date Date, 
isActive int
""")

conn.commit()

In [17]:
cursor.execute ("""
Update customers1 
set end_registration_date = Null, 
isActive = 1 
""")

conn.commit()

In [9]:
cursor.execute("""
Alter table customers1
drop column end_registration_date, isActive
""")

In [20]:
change_mail ='''UPDATE customers1
SET end_registration_date = GETDATE(),
    isActive = 0
WHERE customer_id = ? AND isActive = 1'''
 
id= int(input("enter customer id:"))
cursor.execute(change_mail,(id,))
conn.commit()


 
insert = '''
insert into customers1(customer_id, name, email, phone, address,
   registration_date,
   loyalty_status, country_code,end_registration_date, isActive) values
   (?,'Brad Newton','taylorcatherine@google.net','+1 (537)-674-1158' , '38783 Oliver Street',getdate(),
    'Silver','MT',NULL,1)
'''
 
 
cursor.execute(insert,id)
conn.commit()

In [5]:
#SCD type 3 with Limited History
cursor.execute("""
Alter table customers1
ADD previous_loyalty_status varchar(50)
"""   )

conn.commit()

In [9]:
updating_loyalty = """ 
Update customers1
set previous_loyalty_status = loyalty_status,
    loyalty_status = 'Silver'
WHERE customer_id = ?
"""

id= int(input("enter customer id:"))
cursor.execute(updating_loyalty,(id,))
conn.commit()

In [12]:
#SCD Type 4 Historic Data
cursor.execute("""  
Create Table customers_history(
               customer_id int,
               name varchar(100),
               email varchar(100),
               phone varchar(100),
               address varchar(200),
               registration_date date,
               loyalty_status varchar(20),
               country_code varchar(10),
               archived_on Date
               )
""")

conn.commit()

In [16]:
updating_history = """ 
INSERT INTO customers_history
SELECT customer_id, name, email, phone, address,
       CAST(registration_date AS DATE), loyalty_status, country_code, GETDATE()
FROM customers1
WHERE customer_id = ?
"""

id = int(input("Enter customer id: "))
cursor.execute(updating_history, (id,))
conn.commit()


In [17]:
#updating the main table
updating_main_table = """ 
Update customers1
Set loyalty_status = 'Platinum'
WHERE customer_id = ?
"""

id=int(input("Enter customer_id: "))
cursor.execute(updating_main_table,(id,))
conn.commit()


In [18]:
#SCD Type 6 
cursor.execute(""" 
Alter table customers1 add 
    start_date DATE DEFAULT GETDATE(),
    end_date DATE
""")

conn.commit()

In [19]:
updating_current = """ 
Update customers1 
Set end_date = GetDate(),
isActive = 0
where customer_id = ? and isActive = 1
"""

id = int(input("enter customer_id:"))
cursor.execute(updating_current,(id,))
conn.commit()

In [22]:
inserting_new = """
Insert into customers1(customer_id, name, email, phone, address,
registration_date, loyalty_status, country_code, end_registration_date,
isActive, previous_loyalty_status, start_date, end_date)

Select customer_id, name, 'newemail@example.com', phone, address,
    registration_date, 'Gold', country_code, NULL, 1,
    previous_loyalty_status, GETDATE(), NULL
FROM customers1
WHERE customer_id = ? AND end_date = GETDATE()
"""

id = int(input("enter customer_id:"))
cursor.execute(inserting_new,(id,))
conn.commit()